# 06-post-analysis

How can we verify if the trained models are correct in their predictions? 

One way is to verify against known candidates. 

In [1]:
import sys
sys.path.insert(0, "../")

import pandas as pd
from pathlib import Path

data_dir = Path.cwd().parent.parent.parent / "data"
models_dir = Path.cwd().parent.parent.parent / "models" 

print("Current data directory {}".format(data_dir))

Current data directory /home/oliver/Dokumenter/masterprosjekt/predicting-solid-state-qubit-candidates/data


In [2]:
known_candidates = ["C", "Si", "SiGe", "AlN", "GaN",
                   "AlP", "GaP", "AlAs", "ZnO", "ZnS", "ZnSe", "ZnTe", "CdS"]
QD_2D_candidates = ["BN", "MoS2", "WSe2", #2D
                    "LnAs" #QD
                   ] 

In [3]:
InsertApproach = "01-naive-approach"
numberOfPrincipalComponents = 2

## Training set
How many entries in our training set consists of known candidates?

In [4]:
trainingSet   = pd.read_pickle(data_dir / InsertApproach / "processed" / "trainingSet.pkl")

In [5]:
trainingSet[["full_formula", "pretty_formula", "candidate"]][trainingSet["pretty_formula"].isin(known_candidates)]

,full_formula,pretty_formula,candidate
3,C8,C,1.0
4,C4,C,1.0
5,C2,C,1.0
11,Si2,Si,1.0
12,Si4,Si,1.0
13,C2,C,1.0
29,Al2N2,AlN,0.0
40,Ga2N2,GaN,0.0
50,Zn1Se1,ZnSe,1.0
83,Zn1O1,ZnO,1.0


In [6]:
trainingSet[["full_formula", "pretty_formula", "candidate"]][trainingSet["pretty_formula"].isin(QD_2D_candidates)]

,full_formula,pretty_formula,candidate
20,B4N4,BN,0.0
66,B4N4,BN,0.0
73,W2Se4,WSe2,1.0
110,B2N2,BN,0.0
117,Mo2S4,MoS2,1.0
1238,B1N1,BN,0.0
1312,Mo2S4,MoS2,1.0
1360,B3N3,BN,0.0


## Test set
How many entries in our test set consists of known candidates?

In [7]:
Summary = pd.read_pickle(models_dir / InsertApproach /  "summary" / Path("PCA-" + str(numberOfPrincipalComponents) + "-" + "summary.pkl"))
Summary.shape

(23605, 9)

In [8]:
Summary[Summary["pretty_formula"].isin(known_candidates)]

,material_id,full_formula,pretty_formula,LOG,LOG Prob,RF,RF Prob,GB,GB Prob
1685,mp-370,Cd1S1,CdS,1.0,0.859707,1.0,0.794041,1.0,0.934309
1687,mp-380,Zn2Se2,ZnSe,1.0,0.806741,1.0,0.874402,1.0,0.879188
1724,mp-672,Cd2S2,CdS,1.0,0.800326,1.0,0.874402,1.0,0.879188
1742,mp-830,Ga1N1,GaN,1.0,0.942833,1.0,0.880544,1.0,0.959695
1803,mp-1330,Al1N1,AlN,1.0,0.971800,1.0,0.880544,1.0,0.939438
...,...,...,...,...,...,...,...,...,...
24249,mp-1201781,Zn18S18,ZnS,1.0,0.823328,1.0,0.873766,1.0,0.907013
24311,mp-1202023,Zn18S18,ZnS,1.0,0.820255,1.0,0.873766,1.0,0.907013
24352,mp-1202182,Zn18S18,ZnS,1.0,0.819882,1.0,0.873766,1.0,0.907013
24556,mp-1202959,Zn18S18,ZnS,1.0,0.820089,1.0,0.873766,1.0,0.907013


In [9]:
Summary[Summary["pretty_formula"].isin(QD_2D_candidates)]

,material_id,full_formula,pretty_formula,LOG,LOG Prob,RF,RF Prob,GB,GB Prob
1762,mp-984,B2N2,BN,1.0,0.937980,1.0,0.878453,1.0,0.889943
1819,mp-1434,Mo1S2,MoS2,1.0,0.790853,1.0,0.873766,1.0,0.878086
1840,mp-1639,B1N1,BN,1.0,0.986690,1.0,0.880544,1.0,0.513119
3384,mp-7991,B2N2,BN,1.0,0.932513,1.0,0.875256,1.0,0.922830
4439,mp-13151,B4N4,BN,1.0,0.981644,1.0,0.880544,1.0,0.750406
15144,mp-569655,B4N4,BN,1.0,0.921741,1.0,0.873766,1.0,0.922830
15929,mp-604884,B2N2,BN,1.0,0.944885,1.0,0.878453,1.0,0.889943
16269,mp-629015,B2N2,BN,1.0,0.944437,1.0,0.880544,1.0,0.889943
16647,mp-644751,B4N4,BN,1.0,0.979202,1.0,0.880544,1.0,0.750406
19883,mp-1060281,B1N1,BN,1.0,0.920186,1.0,0.875256,1.0,0.930395


## How many entries does the models agree on? 

In [10]:
Summary[(Summary["RF "] == 1) & 
        (Summary["GB "] == 1) & 
        (Summary["LOG "] == 1)]

,material_id,full_formula,pretty_formula,LOG,LOG Prob,RF,RF Prob,GB,GB Prob
1649,mvc-12905,Fe4O8,FeO2,1.0,0.884850,1.0,0.873766,1.0,0.922830
1650,mp-25,N8,N2,1.0,0.845749,1.0,0.873766,1.0,0.907664
1651,mp-111,Ne1,Ne,1.0,0.868613,1.0,0.875256,1.0,0.907013
1652,mp-118,P24,P,1.0,0.714305,1.0,0.788236,1.0,0.645889
1653,mp-154,N8,N2,1.0,0.844523,1.0,0.873766,1.0,0.907664
...,...,...,...,...,...,...,...,...,...
25249,mp-1539137,Rb1Cr5S8,RbCr5S8,1.0,0.800309,1.0,0.873766,1.0,0.895776
25250,mp-1541522,Bi2P2O8,BiPO4,1.0,0.872061,1.0,0.873766,1.0,0.930395
25251,mp-1541714,Cs3As2Cl9,Cs3As2Cl9,1.0,0.716677,1.0,0.838583,1.0,0.853507
25252,mp-1542038,Cs2Sn2Se6,CsSnSe3,1.0,0.569222,1.0,0.769221,1.0,0.576305
